In [4]:
!pip install huggingface_hub --upgrade -q



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
!git branch -a


* main


## Если скачивать локально

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="Qwen/Qwen3-4B-Instruct-2507", local_dir="./Qwen3-4B-Instruct-2507")
model = AutoModelForCausalLM.from_pretrained(
   './gemma-3-12b-it',
    torch_dtype="auto",
    device_map="auto",
    local_files_only=True,
    trust_remote_code=True

).eval()
tokenizer = AutoTokenizer.from_pretrained('./Qwen3-4B-Instruct-2507')
# ЕСЛИ ЛОКАЛЬОНО ЧЕРЕЗ HF_HUB

## Если нелокально

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
   'Qwen/Qwen3-4B-Instruct-2507',
    torch_dtype="auto",
    device_map="auto",

).eval()
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-4B-Instruct-2507')
# ЕСЛИ НЕЛОКАЛЬНО!

## ЕСЛИ AUTOAWQ, то не забудь pip install awq (не везде работает, например в kaggle не работает)

In [ ]:
!pip install autoawq -q
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model = AutoModelForCausalLM.from_pretrained(
   'Qwen/Qwen2.5-7B-Instruct-AWQ',
    torch_dtype="auto",
    device_map="auto",
).eval()
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-7B-Instruct-AWQ')

## Пишем промпт/получаем ответ

In [1]:
prompt = "what's 2 + 2?"
messages = {'role': 'user', 'content': prompt}
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors='pt').to(model.device)



NameError: name 'tokenizer' is not defined

In [ ]:
with torch.no_grad():
    gen_ids = model.generate(
        **model_inputs,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        top_k=50
    )

In [ ]:
# 1. Жадный поиск (детерминированный)
model.generate(do_sample=False)
# Результат: "Погода сегодня хорошая."

# 2. Случайная выборка с температурой
model.generate(do_sample=True, temperature=0.7)
# Результат: "Погода сегодня прекрасная." (вариант 1)
# Результат: "Погода сегодня замечательная." (вариант 2)

# 3. Только top-p выборка
model.generate(do_sample=True, top_p=0.9)
# Исключает маловероятные варианты

# 4. Комбинированная стратегия
model.generate(do_sample=True, temperature=0.5, top_p=0.9, top_k=50)
# Баланс между качеством и разнообразием

In [ ]:
output_ids = gen_ids[0][len(model_inputs.input_ids[0]):].tolist() 
answer = tokenizer.decode(output_ids, skip_special_tokens=True)


## Экстрактор ответа пишешь сам, но посмотри на выводы моделй в конце

In [ ]:
# скорее всего делай что-то такое
last_string = answer.split('\n')
# было и такое:
another = []
for q in answers_by_model:
    q = q.split('\n')[-1].split(':')[-1].replace(' ', '')
    another.append(q)


NameError: name 'answer' is not defined

# Целый код для копирования

In [6]:
import numpy as np
import pandas as pd 
import os
import pickle
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = 'Qwen/Qwen3-4B-Instruct-2507'
model = AutoModelForCausalLM.from_pretrained(
   model_path,
    torch_dtype="auto",
    device_map="auto",
).eval()
tokenizer = AutoTokenizer.from_pretrained(model_path)

with open("input.pickle", "rb") as f:
    dataset = pickle.load(f) 

def get_an_answer(question, options):
    prompt = f""" 
    You're an expert in maths and physics and you're given some tasks
    ###
    These tasks are about math and physics ### You should complete them, choosing several options or one option.
    You better reason step by step.
    Remember, correct answers may have several options
    ###
    At the end YOU SHOULD WRITE JUST THE LETTER ANSWER, NOT A NUMBER OR etc.,
    ONLY LETTER OF THE ANSWER WITHOUT ANY SPACE OR COMMA: Example: AB, WITHOUT SPACE, NOT 'A B'!
    WITHOUT ANY SEPARATOR if none of them are true, write just AB
    ### Question: {question}
    Options: {options} """
    

    messages = [{'role': 'user', 'content': prompt}]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    with torch.no_grad():
        gen_ids = model.generate(
            **model_inputs,
            max_new_tokens=1500, 
            do_sample=True,      
            top_k=50,          
            top_p=0.95,      
            temperature=0.3 
            
        )
    output_ids = gen_ids[0][len(model_inputs.input_ids[0]):].tolist() 
    answer = tokenizer.decode(output_ids, skip_special_tokens=True)

    return answer

answers_by_model = []

for i in range(len(dataset)):
    answer = get_an_answer(dataset[i]['question'], dataset[i]['options'])
    answers_by_model.append(answer)
    print(f'Done: {i + 1}')

: 